In [1]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # use GPU
# Using GPU during inference has deterministic results (same as CPU)

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs import utils, mnist32_cnn
from libs.constants import model_seeds
from libs.fitnessfns import ff_mnist32_cnn_ERR
from timeit import default_timer as timer

In [2]:
# set seed
this_seed = 9387
tf.random.set_seed(this_seed)
np.random.seed(this_seed)
random.seed(this_seed)
os.environ['PYTHONHASHSEED']=str(this_seed)

In [3]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
# Prepare dataset
# Use only test images
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)
test_images = test_images.astype(np.float32) / 255.0  
test_images = np.expand_dims(test_images, axis=-1)
# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
test_images = tf.image.resize(test_images, [image_x_size, image_y_size])

In [5]:
# Load test set
# Testing with only im_num images
im_num = 128*40#128*40=5120 #[128*78 = 9984]
test_set = (test_images[im_num:], 
            test_labels[im_num:])

In [6]:
# Batchsize for evaluation
EVAL_BATCHSIZE = 128

In [7]:
def eval_error_injected_no_shuffle(model, 
                                  error_layer, 
                                  error_profile, 
                                  ERR_PARAM,
                                  test_set,
                                  batchsize):
    
    error_profile_c0=None
    error_profile_h0=None
    error_profile_h1=None
    error_profile_h2=None
    error_profile_op=None
    if error_layer == "c0":
        error_profile_c0=error_profile
    elif error_layer == "h0":
        error_profile_h0=error_profile
    elif error_layer =="h1":
        error_profile_h1=error_profile
    elif error_layer == "h2":
        error_profile_h2=error_profile
    elif error_layer == "op":
        error_profile_op=error_profile
    elif error_layer == "all":
        error_profile_c0=error_profile
        error_profile_h0=error_profile
        error_profile_h1=error_profile
        error_profile_h2=error_profile
        error_profile_op=error_profile
        
    shuffle_order_c0 = None
    shuffle_order_h0 = None
    shuffle_order_h1 = None
    shuffle_order_h2 = None
    shuffle_order_op = None
        
    acc_list = []
    for _ in range(3):
        accuracy = ff_mnist32_cnn_ERR(model,
                                        error_profile_c0,
                                        error_profile_h0,
                                        error_profile_h1,
                                        error_profile_h2,
                                        error_profile_op,
                                        ERR_PARAM,
                                        shuffle_order_c0,
                                        shuffle_order_h0,
                                        shuffle_order_h1,
                                        shuffle_order_h2,
                                        shuffle_order_op,
                                        test_set=test_set,
                                        batchsize=batchsize)
        acc_list.append(accuracy)
    return np.mean(acc_list)

In [8]:
from libs.constants import error_seeds
error_profile_tag_list = []
MAX_FAULT_PROB_LIST = [1E-3, 2E-3, 5E-3]
for MAX_FAULT_PROB in MAX_FAULT_PROB_LIST:
    for seed in error_seeds:
        err_tag = "LIM_" + f"{int(MAX_FAULT_PROB*1E3):02d}" + "-" +str(seed)
        error_profile_tag_list.append(err_tag)
# error_profile_tag_list

In [9]:
model_type = "mnist32-cnn_1024_256_64"
results = {}
for model_seed in  model_seeds:
    results[model_seed] = {}
    # Get model
    model_instance = model_type + "-" + str(model_seed)
    dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
    model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
    model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
    model_filename = model_instance + ".h5"
    model_file = pathlib.Path(model_folder/ model_filename)
    # Load model
    model = tf.keras.models.load_model(model_file)
    print(model_instance)
    
    # Original Accuracy
    original_accuracy = ff_mnist32_cnn_ERR(model,
                                            error_profile_c0=None,
                                            error_profile_h0=None,
                                            error_profile_h1=None,
                                            error_profile_h2=None,
                                            error_profile_op=None,
                                            ERR_PARAM=None,
                                            shuffle_order_c0=None,
                                            shuffle_order_h0=None,
                                            shuffle_order_h1=None,
                                            shuffle_order_h2=None,
                                            shuffle_order_op=None,
                                            test_set=test_set,
                                            batchsize=EVAL_BATCHSIZE)
    print("ORIGINAL ACC: ", np.around(original_accuracy, decimals=4))
    results[model_seed]["original_acc"] = np.around(original_accuracy, decimals=4)
    
    
    # Error Injection
    for error_profile_tag in error_profile_tag_list:
        print(error_profile_tag)
        results[model_seed][error_profile_tag] = {}
        # Load error profile
        err_profile_filename = "../error_profiles/"+ error_profile_tag + ".npy"
        error_profile = np.load(err_profile_filename)

        ERR_PARAM_list = [1,-1]
        error_layer_list = ["c0", "h0", "h1", "h2", "op", "all"]
        for ERR_PARAM in ERR_PARAM_list:
            results[model_seed][error_profile_tag][ERR_PARAM]={}
            for error_layer in error_layer_list:
                eval_acc = eval_error_injected_no_shuffle(model, 
                                                          error_layer, 
                                                          error_profile, 
                                                          ERR_PARAM,
                                                          test_set,
                                                          EVAL_BATCHSIZE)
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer] = np.around(eval_acc,decimals=4)
                results_filename = 'fault_injection_analysis_results-mnist32_cnn--lowERR--ERR1_ERR-1.npy'
                np.save(results_filename, results) 
                print(ERR_PARAM, '\t' ,error_layer, '\t' , np.around(eval_acc,decimals=4))
            print('*'*50)    
        print()

mnist32-cnn_1024_256_64-1023
ORIGINAL ACC:  0.9934
LIM_01-2188
1 	 c0 	 0.9086
1 	 h0 	 0.9931
1 	 h1 	 0.9934
1 	 h2 	 0.9852
1 	 op 	 0.9916
1 	 all 	 0.8996
**************************************************
-1 	 c0 	 0.8663
-1 	 h0 	 0.9934
-1 	 h1 	 0.9935
-1 	 h2 	 0.9855
-1 	 op 	 0.9911
-1 	 all 	 0.8555
**************************************************

LIM_01-3987
1 	 c0 	 0.8394
1 	 h0 	 0.9933
1 	 h1 	 0.9934
1 	 h2 	 0.9855
1 	 op 	 0.9913
1 	 all 	 0.8289
**************************************************
-1 	 c0 	 0.8457
-1 	 h0 	 0.9931
-1 	 h1 	 0.9934
-1 	 h2 	 0.9849
-1 	 op 	 0.991
-1 	 all 	 0.8335
**************************************************

LIM_01-4981
1 	 c0 	 0.864
1 	 h0 	 0.9934
1 	 h1 	 0.9933
1 	 h2 	 0.9851
1 	 op 	 0.9915
1 	 all 	 0.8563
**************************************************
-1 	 c0 	 0.8401
-1 	 h0 	 0.9934
-1 	 h1 	 0.9932
-1 	 h2 	 0.9842
-1 	 op 	 0.9916
-1 	 all 	 0.8278
**************************************************

LIM_01